In [3]:
pip install tqdm

  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.1/7.1 MB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 34.1/34.1 MB 5.7 MB/s eta 0:00:00
     -------------------------------------- 307.0/307.0 KB 9.6 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=2f87b7fbb48ed909cd539777a50daa6edba5fbf6cf3bf0b0358801b2cb8520c1
  Stored in directory: c:\users\vishal j. soni\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

     ---------------------------------------- 7.2/7.2 MB 4.2 MB/s eta 0:00:00
     -------------------------------------- 895.5/895.5 KB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 51.6/51.6 KB ? eta 0:00:00
     ---------------------------------------- 3.2/3.2 MB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
# METRICS

import math
import numpy as np

def accuracy(y_hat, y):
    """
    Function to calculate the accuracy

    Inputs:
    > y_hat: pd.Series of predictions
    > y: pd.Series of ground truth
    Output:
    > Returns the accuracy as float
    """
    """
    The following assert checks if sizes of y_hat and y are equal.
    Students are required to add appropriate assert checks at places to
    ensure that the function does not fail in corner cases.
    """
    assert(y_hat.size == y.size)
    # TODO: Write here
    True_Positive = 0
    Total = y_hat.size
    for i in range(y.size):
        if y_hat[i] == y[i]:
            True_Positive += 1
    return float(True_Positive) / Total
    pass

def precision(y_hat, y, cls):
    """
    Function to calculate the precision

    Inputs:
    > y_hat: pd.Series of predictions
    > y: pd.Series of ground truth
    > cls: The class chosen
    Output:
    > Returns the precision as float
    """
    assert(y_hat.size == y.size)
    True_Positive = 0
    Total_Predicted_Positive = 0
    for i in range(y.size):
        if y_hat[i] == cls and y[i] == cls:
            True_Positive += 1
        if y_hat[i] == cls:
            Total_Predicted_Positive += 1
    return float(True_Positive) / Total_Predicted_Positive
    pass

def recall(y_hat, y, cls):
    """
    Function to calculate the recall

    Inputs:
    > y_hat: pd.Series of predictions
    > y: pd.Series of ground truth
    > cls: The class chosen
    Output:
    > Returns the recall as float
    """
    assert(y_hat.size == y.size)
    Actual_Positives = 0
    Correctly_Predicted_Positive = 0
    for i in range(y.size):
        if y[i] == cls and y_hat[i] == cls:
            Correctly_Predicted_Positive += 1
        if y[i] == cls:
            Actual_Positives += 1
    return float(Correctly_Predicted_Positive) / Actual_Positives
    pass

def rmse(y_hat, y):
    """
    Function to calculate the root-mean-squared-error(rmse)

    Inputs:
    > y_hat: pd.Series of predictions
    > y: pd.Series of ground truth
    Output:
    > Returns the rmse as float
    """
    assert(y_hat.size == y.size)
    sq = 0;
    for i in range(y.size):
        sq += (y[i] - y_hat[i]) ** 2
    mean_sq = sq / y.size
    return float(math.sqrt(float(mean_sq)))
    pass

def mae(y_hat, y):
    """
    Function to calculate the mean-absolute-error(mae)

    Inputs:
    > y_hat: pd.Series of predictions
    > y: pd.Series of ground truth
    Output:
    > Returns the mae as float
    """
    assert(y_hat.size == y.size)
    abs_sum = 0
    for i in range(y.size):
        abs_sum = abs(y[i] - y_hat[i])
    return float(abs_sum) / y.size
    pass


In [70]:
# BAGGING.py

# from tree.base import DecisionTree
import numpy as np
import pandas as pd
from sklearn import tree as sktree
import matplotlib.pyplot as plt
from sklearn.utils.extmath import weighted_mode
from sklearn.tree import DecisionTreeClassifier
import os

class BaggingClassifier():
    def __init__(self, base_estimator = DecisionTreeClassifier, n_estimators=100, max_depth=100, criterion="entropy"):
        '''
        :param base_estimator: The base estimator model instance from which the bagged ensemble is built (e.g., DecisionTree(), LinearRegression()).
                               You can pass the object of the estimator class
        :param n_estimators: The number of estimators/models in ensemble.
        '''
        self.base_estimator = base_estimator # SKtree
        self.max_depth = max_depth 
        self.trees = []
        self.samples_xy = []
        self.criterion = criterion
        self.n_estimators = n_estimators # number of iterations/ estimators for which we will run bagging.

        pass

    def fit(self, X, y):
        """
        Function to train and construct the BaggingClassifier
        Inputs:
        X: pd.DataFrame with rows as samples and columns as features (shape of X is N X P) where N is the number of samples and P is the number of columns.
        y: pd.Series with rows corresponding to output variable (shape of Y is N)
        """
        # getting data samples and learning models for all those samples:
        for iter in range(self.n_estimators):
            curr_X = X.sample(frac=1, axis='rows', replace=True) # sampling with replacement
            curr_y = y[curr_X.index]

            curr_X = curr_X.reset_index(drop=True) # maininting new indices
            curr_y = curr_y.reset_index(drop=True)

            tree = self.base_estimator(criterion=self.criterion) # learning new model for current sample
            tree.fit(curr_X, curr_y) # fitting in sktree

            self.trees.append(tree) # storing the tree corresponding data publicly
            self.samples_xy.append([curr_X, curr_y])
        pass

    def predict(self, X):
        """
        Funtion to run the BaggingClassifier on a data point
        Input:
        X: pd.DataFrame with rows as samples and columns as features
        Output:
        y: pd.Series with rows corresponding to output variable. THe output variable in a row is the prediction for sample in corresponding row in X.
        """
        Predicted = None
        i = 0
        for tree in self.trees:
            if Predicted is not None:
                Predicted[i] = tree.predict(X)
            else:
                Predicted = pd.Series(tree.predict(X)).to_frame()
            i += 1
#         print(Predicted)
#         print(Predicted.mode(axis = 1))
        best_prediction = Predicted.mode(axis = 1)
#         print(best_prediction[0])
        return best_prediction[0]
        pass

    def plot(self, X, y):
        """
        Function to plot the decision surface for BaggingClassifier for each estimator(iteration).
        Creates two figures
        Figure 1 consists of 1 row and `n_estimators` columns and should look similar to slide #16 of lecture
        The title of each of the estimator should be iteration number

        Figure 2 should also create a decision surface by combining the individual estimators and should look similar to slide #16 of lecture

        Reference for decision surface: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

        This function should return [fig1, fig2]

        """
        color = ["y", "r", "b"]
        Z_ls = []
        fig1, ax1 = plt.subplots(
            1, len(self.trees), figsize=(5*len(self.trees), 4))

        x_min, x_max = X[0].min(), X[0].max()
        x_range = x_max-x_min
        y_min, y_max = X[1].min(), X[1].max()
        y_range = y_max-y_min
#         print(x_range)
#         print(y_range)

        # plotting surfaces for all the sampling iterations:
        for i, tree in enumerate(self.trees):
            X_tree, y_tree = self.samples_xy[i]

            xx, yy = np.meshgrid(np.arange(x_min-0.5, x_max+0.5, (x_range)/50),
                                 np.arange(y_min-0.5, y_max+0.5, (y_range)/50))
            
            ax1[i].set_xlabel("X1")
            ax1[i].set_ylabel("X2")
            Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
            Z = Z.reshape(xx.shape)
            Z_ls.append(Z)
            c_surf = ax1[i].contourf(xx, yy, Z, cmap=plt.cm.YlOrBr)
            fig1.colorbar(c_surf, ax=ax1[i])

            for y_label in y.unique():
                idx = y_tree == y_label
                id = list(y_tree.cat.categories).index(y_tree[idx].iloc[0])
                ax1[i].scatter(X_tree.loc[idx, 0], X_tree.loc[idx, 1], c=color[id],
                               cmap=plt.cm.YlOrBr, s=30,
                               label="Class_Type: "+str(y_label))
            ax1[i].legend()
            ax1[i].set_title("Decision Tree Surface: {}".format(i + 1))
            

        # plottinh common surface
        fig2, ax2 = plt.subplots(1, 1, figsize=(5, 4))
        Z_ls = np.array(Z_ls)
        Common_surf, bleh = weighted_mode(Z_ls, np.ones(Z_ls.shape))
        c_surf = ax2.contourf(xx, yy, Z, cmap=plt.cm.YlOrBr)
        for y_label in y.unique():
            idx = y == y_label
            id = list(y.cat.categories).index(y[idx].iloc[0])
            ax2.scatter(X.loc[idx, 0], X.loc[idx, 1], c=color[id],
                        cmap=plt.cm.YlOrBr, s=30,
                        label="Class_Type: "+str(y_label))
        ax2.set_xlabel("X1")
        ax2.set_ylabel("X2")
        ax2.legend()
        ax2.set_title("Common Decision Surface")
        fig2.colorbar(c_surf, ax=ax2)

        fig1.savefig("Q6_fig1.png")
        fig2.savefig("Q6_fig2.png")
        return fig1, fig2
        pass


In [71]:
# Q6_BAGGING.py

"""
The current code given is for the Assignment 2.
> Classification
> Regression
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from metrics import *

# from ensemble.bagging import BaggingClassifier
# from tree.base import DecisionTree
# Or use sklearn decision tree
from sklearn.tree import DecisionTreeClassifier
# from linearRegression.linearRegression import LinearRegression

########### BaggingClassifier ###################

N = 30
P = 2
NUM_OP_CLASSES = 2
n_estimators = 3
X = pd.DataFrame(np.abs(np.random.randn(N, P)))
y = pd.Series(np.random.randint(NUM_OP_CLASSES, size = N), dtype="category")

criteria = 'information_gain'
tree = DecisionTreeClassifier(criterion=criteria)
Classifier_B = BaggingClassifier(base_estimator=DecisionTreeClassifier, n_estimators=n_estimators )
Classifier_B.fit(X, y)
y_hat = Classifier_B.predict(X)
[fig1, fig2] = Classifier_B.plot(X, y)
print('Criteria :', criteria)
print('Accuracy: ', accuracy(y_hat, y))
for cls in y.unique():
    print('Precision: ', precision(y_hat, y, cls))
    print('Recall: ', recall(y_hat, y, cls))


KeyError: 'information_gain'